<a href="https://colab.research.google.com/github/shivang-niranjan/Hyperspectral-imaging/blob/main/Shivang_Project_2_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Load your dataset with error handling
df = pd.read_csv('/content/news.csv', on_bad_lines='skip')
df = df[['text', 'label']]  # Ensure you have 'text' and 'label' columns

# Check for missing values
print("Missing values:", df.isnull().sum())

# Drop missing values if any
df.dropna(inplace=True)

# Preprocess labels
le = LabelEncoder()
df['label'] = le.fit_transform(df['label'])

# Split data
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

# Tokenize text
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

# Pad sequences
maxlen = 100  # Maximum length of a sequence
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

# Build the model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=maxlen))
model.add(LSTM(128, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test), verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f'Test Accuracy: {accuracy*100:.2f}%')


Missing values: text     0
label    0
dtype: int64
Epoch 1/5
80/80 [==============================] - 27s 302ms/step - loss: 0.6271 - accuracy: 0.6433 - val_loss: 0.4541 - val_accuracy: 0.8003
Epoch 2/5
80/80 [==============================] - 26s 331ms/step - loss: 0.3045 - accuracy: 0.8721 - val_loss: 0.3589 - val_accuracy: 0.8445
Epoch 3/5
80/80 [==============================] - 23s 288ms/step - loss: 0.1823 - accuracy: 0.9455 - val_loss: 0.4966 - val_accuracy: 0.8145
Epoch 4/5
80/80 [==============================] - 24s 299ms/step - loss: 0.2175 - accuracy: 0.9298 - val_loss: 0.4684 - val_accuracy: 0.8493
Epoch 5/5
40/40 [==============================] - 4s 101ms/step - loss: 0.4499 - accuracy: 0.8556
Test Accuracy: 85.56%
